In [2]:
from fh_id_data import fh_id_data
from fh_ca import campo_amostral
from fh_filter import get_filter
import os
from supabase import create_client, Client
import math
table = 'table_statics'
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

def mout_filters(fh_q_dict, dict_campo_amostral, category):
    dict_filter_over = {}
    dict_filter_under = {}
    ca_category = dict_campo_amostral[category]
    if category == 'gols':
        atual = fh_q_dict['gols']['sum_period1']
        for i in range(1,6):
            dict_filter_over[i] = [atual + i , get_filter(fh_q_dict, category, i),0,0]
            dict_filter_under[i] = [atual + i, ca_category - get_filter(fh_q_dict, category, i),0, 0]

    elif category == 'corners':
        atual = fh_q_dict['corners']['sum_1st_cornerkicks'][2]
        for i in range(1,6):
            dict_filter_over[i] = [atual + i + 2, get_filter(fh_q_dict, category, i + 2),0,0]
            dict_filter_under[i] = [atual + i + 2, ca_category - get_filter(fh_q_dict, category, i + 2),0, 0]

    elif category == 'yellowcards':
        atual = fh_q_dict['yellowcards']['sum_1st_yellowcards'][2]
        for i in range(1,6):
            dict_filter_over[i] = [atual + i , get_filter(fh_q_dict, category, i + 2),0,0]
            dict_filter_under[i] = [atual + i, ca_category - get_filter(fh_q_dict, category, i + 2),0, 0]

    return dict_filter_over, dict_filter_under

 
def mount_resp(dict_filter_over, dict_filter_under,dict_campo_amostral, category):
    for i in range(1,len(dict_filter_over) + 1):
        prob = (dict_filter_over[i][1] * 100) / dict_campo_amostral[category]
        dict_filter_over[i][2] = prob
        odd = 1/(prob/100)
        dict_filter_over[i][3] = odd

    for i in range(1,len(dict_filter_under) + 1):
        prob = (dict_filter_under[i][1] * 100) / dict_campo_amostral[category]
        dict_filter_under[i][2] = prob
        odd = 1/(prob/100)
        dict_filter_under[i][3] = odd

    return dict_filter_over, dict_filter_under


def over_under(event_id):
    dict_resp = {}
    event_info = fh_id_data(event_id)
    list_resp = [event_info]
    dict_campo_amostral, fh_q_dict = campo_amostral(event_id)
    
    category = 'gols'
    dict_filter_over, dict_filter_under = mout_filters(fh_q_dict, dict_campo_amostral, category)
    resp_over, resp_under = mount_resp(dict_filter_over, dict_filter_under,dict_campo_amostral, category)
    dict_resp = {
        'gols': [resp_over, resp_under],
    }
    list_resp.append(dict_resp)

    category = 'corners'
    dict_filter_over, dict_filter_under = mout_filters(fh_q_dict, dict_campo_amostral, category)
    resp_over, resp_under = mount_resp(dict_filter_over, dict_filter_under,dict_campo_amostral, category)
    dict_resp = {
        'corners': [resp_over, resp_under],
    }
    list_resp.append(dict_resp)

    category = 'yellowcards'
    dict_filter_over, dict_filter_under = mout_filters(fh_q_dict, dict_campo_amostral, category)
    resp_over, resp_under = mount_resp(dict_filter_over, dict_filter_under,dict_campo_amostral, category)
    dict_resp = {
        'yellowcards': [resp_over, resp_under],
    }
    list_resp.append(dict_resp)

    return list_resp


In [3]:
event_id = 13472618
# dict_campo_amostral, fh_q_dict = campo_amostral(event_id)
# event_info = fh_id_data(event_id)
# dict_campo_amostral, fh_q_dict

In [4]:
over_under_resp = over_under(event_id) 
over_under_resp

[{'event_id': 13472618,
  'id_tournament': 'Brasileirão Betano',
  'home_name': 'Ceará',
  'away_name': 'Cruzeiro',
  'home_score': 0,
  'awayScore': 0,
  'home_statistics': {'Ball possession': 40,
   'Expected goals': 0.36,
   'Total shots': 6,
   'Shots on target': 2,
   'Shots inside box': 4,
   'Final third entries': 27,
   'cornerKicks': 1,
   'fouls': 6,
   'yellowCards': 0},
  'away_statistics': {'Ball possession': 60,
   'Expected goals': 0.44,
   'Total shots': 7,
   'Shots on target': 2,
   'Shots inside box': 1,
   'Final third entries': 31,
   'cornerKicks': 4,
   'fouls': 6,
   'yellowCards': 2}},
 {'gols': [{1: [1, 95, 81.89655172413794, 1.2210526315789474],
    2: [2, 60, 51.724137931034484, 1.9333333333333331],
    3: [3, 24, 20.689655172413794, 4.833333333333333],
    4: [4, 6, 5.172413793103448, 19.333333333333332],
    5: [5, 1, 0.8620689655172413, 116.0]},
   {1: [1, 21, 18.103448275862068, 5.523809523809525],
    2: [2, 56, 48.275862068965516, 2.0714285714285716],


In [5]:
display(over_under_resp[0]['home_name'])
display(over_under_resp[0]['away_name'])
display(over_under_resp[1]['gols'][0])
display(over_under_resp[1]['gols'][1])

'Ceará'

'Cruzeiro'

{1: [1, 95, 81.89655172413794, 1.2210526315789474],
 2: [2, 60, 51.724137931034484, 1.9333333333333331],
 3: [3, 24, 20.689655172413794, 4.833333333333333],
 4: [4, 6, 5.172413793103448, 19.333333333333332],
 5: [5, 1, 0.8620689655172413, 116.0]}

{1: [1, 21, 18.103448275862068, 5.523809523809525],
 2: [2, 56, 48.275862068965516, 2.0714285714285716],
 3: [3, 92, 79.3103448275862, 1.2608695652173914],
 4: [4, 110, 94.82758620689656, 1.0545454545454545],
 5: [5, 115, 99.13793103448276, 1.008695652173913]}